In [2]:
import numpy as np
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras import Input
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models

path = "datasets2022.npz"
data = np.load(path)
X_train = data['train']
y_train = data['y_train']
X_test = data['test']
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, shuffle=False)
X_val1 = X_val.copy()

## Preprocessing

In [203]:
import imutils
from PIL import ImageFilter
from PIL import Image

# Rotate the image for 90,180,270 degrees
rotated_X = [imutils.rotate(x, angle=90) for x in X_train]
rotated_X += [imutils.rotate(x, angle=180) for x in X_train]
rotated_X += [imutils.rotate(x, angle=270) for x in X_train]
rotated_X = np.expand_dims(np.array(rotated_X), axis=3)

flip1_X = X_train[:,::-1,:,:]
flip2_X = X_train[:,:,::-1,:]
X_train  = np.concatenate([X_train, rotated_X, flip1_X, flip2_X], axis=0)
y_train = np.concatenate([y_train]*6, axis=0)

X_train1 = X_train.copy() 
X_train1, y_train1 = shuffle(X_train1, y_train, random_state=42)

X_train = X_train.squeeze()

# Mode filter
for i in range(X_train.shape[0]):
    img = Image.fromarray(X_train[i]*255)
    img = img.convert('RGB')
    img_modefilter=img.filter(ImageFilter.ModeFilter(2))
    img = img_modefilter.convert('F')
    X_train[i] = np.array(img)/255
X_train = np.expand_dims(np.array(X_train), axis=3)
X_train, y_train = shuffle(X_train, y_train, random_state=42)

del rotated_X
del flip1_X
del flip2_X
del data


## Model

### Model 1: Dropout

In [ ]:
def build_model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                    input_shape=(52,52,1), data_format='channels_last',
                    activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))  
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())  
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(84, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='sigmoid'))
    model.compile(optimizer = "adam",
                    loss = "binary_crossentropy",
                    metrics = ['binary_accuracy'])
    return model


In [ ]:
import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=2)
model = build_model()
model.fit(X_train, y_train, validation_data = [X_val, y_val], batch_size=128, epochs=20,callbacks=[callback])

### Model 2: Dropblock

In [ ]:
from keras_drop_block import DropBlock2D
def build_model1():
    model = Sequential()
    model.add(DropBlock2D(block_size=7, keep_prob=0.7, name='Input-Dropout1'))
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                    input_shape=(52,52,1), data_format='channels_last',
                    activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(DropBlock2D(block_size=7, keep_prob=0.7, name='Input-Dropout2'))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))  
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(DropBlock2D(block_size=7, keep_prob=0.7, name='Input-Dropout3'))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())  
    model.add(Dense(128, activation='tanh'))
    model.add(Dense(84, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='sigmoid'))
    model.compile(optimizer = "adam",
                    loss = "binary_crossentropy",
                    metrics = ['binary_accuracy'])
    return model

In [ ]:
model1 = build_model1()
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=2)
model1.fit(X_train, y_train,batch_size=128,validation_data =[X_val,y_val], epochs=200,callbacks=[callback])

### Model 3: VGGNet

In [ ]:
import torch
from torch.nn import LazyBatchNorm2d, Sigmoid
def VGGLayers1(configs):
    channels = 1 
    layers = []
    for (N, C) in configs:
        for _ in range(N):
            conv = torch.nn.Conv2d(channels, C, kernel_size=6, padding=3)
            layers += [conv, torch.nn.ReLU(inplace=True)]
            channels = C
        layers += [torch.nn.MaxPool2d(kernel_size=5, stride=2)]
        layers += [LazyBatchNorm2d()]
    return torch.nn.Sequential(*layers)

def VGGLayers2(configs):
    channels = 32 
    layers = []
    for (N, C) in configs:
        for _ in range(N):
            conv = torch.nn.Conv2d(channels, C, kernel_size=4, padding=1)
            layers += [conv, torch.nn.ReLU(inplace=True)]
            channels = C
        layers += [torch.nn.MaxPool2d(kernel_size=3, stride=2)]
    return torch.nn.Sequential(*layers)

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.VGG1 = VGGLayers1(configs=[(2,32)])
        self.VGG2 = VGGLayers2(configs=[(2,32),(2,3)])
        self.fc1 = torch.nn.Linear(20000, 300)
        self.fc2 = torch.nn.Linear(300, 200)
        self.fc3 = torch.nn.Linear(300, 8)
        self.droupout = torch.nn.Dropout(p=0.5)
        self.fla = Flatten()
        self.sig = Sigmoid()
    def forward(self, x):
        out = self.VGG1(x)
#         out = self.VGG2(out)
        out = self.fla(out)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.droupout(out)
#         out = self.fc2(out)
#         out = F.relu(out)
        out = self.droupout(out)
        out = self.fc3(out)
        out = self.sig(out)  
        return out

# Evaluation

In [ ]:
y_train_prob = model.predict(X_train)
y_val_prob = model.predict(X_val)
y_test_prob = model.predict(X_test)

## With Mode Filter

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, hamming_loss

def round_prob(y_prob):
    return np.round(y_prob).astype(int)
y_train_pred, y_val_pred, y_test_pred = round_prob(y_train_prob), round_prob(y_val_prob), round_prob(y_test_prob)

print("train accuracy:         ", accuracy_score(y_train_pred, y_train))
print("validation accuracy:    ", accuracy_score(y_val_pred, y_val))
print("validation precision:   ", np.round(precision_score(y_val, y_val_pred, average=None),4)) # true +ve / pred +ve
print("validation recall:      ", np.round(recall_score(y_val, y_val_pred, average=None),4))    # true +ve / all +ve
print("validation hamming loss:", hamming_loss(y_val_pred, y_val))

## Without Mode Filter

In [ ]:
model = build_model()
model.fit(X_train1, y_train1, validation_data = [X_val1, y_val], batch_size=128, epochs=20,callbacks=[callback])
y_train_prob = model.predict(X_train1)
y_val_prob = model.predict(X_val1)
y_test_prob = model.predict(X_test)
y_train_pred, y_val_pred, y_test_pred = round_prob(y_train_prob), round_prob(y_val_prob), round_prob(y_test_prob)
print("train accuracy:         ", accuracy_score(y_train_pred, y_train))
print("validation accuracy:    ", accuracy_score(y_val_pred, y_val))
print("validation precision:   ", np.round(precision_score(y_val, y_val_pred, average=None),4)) # true +ve / pred +ve
print("validation recall:      ", np.round(recall_score(y_val, y_val_pred, average=None),4))    # true +ve / all +ve
print("validation hamming loss:", hamming_loss(y_val_pred, y_val))

## Mode Filter + Dropblock

In [ ]:
y_train_prob1 = model1.predict(X_train)
y_val_prob1 = model1.predict(X_val)
y_test_prob1 = model1.predict(X_test)
y_train_pred1, y_val_pred1,y_test_prob1 = round_prob(y_train_prob1), round_prob(y_val_prob1), round_prob(y_test_prob1)
print("train accuracy:         ", accuracy_score(y_train_pred1, y_train))
print("validation accuracy:    ", accuracy_score(y_val_pred1, y_val))
print("validation precision:   ", np.round(precision_score(y_val, y_val_pred1, average=None),4)) # true +ve / pred +ve
print("validation recall:      ", np.round(recall_score(y_val, y_val_pred1, average=None),4))    # true +ve / all +ve
print("validation hamming loss:", hamming_loss(y_val_pred1, y_val))

## Mode Filter w/o Dropblock

In [ ]:
model = build_model()
model.fit(X_train1, y_train1, validation_data = [X_val1, y_val], batch_size=128, epochs=20,callbacks=[callback])
y_train_prob = model.predict(X_train1)
y_val_prob = model.predict(X_val1)
y_test_prob = model.predict(X_test)
y_train_pred, y_val_pred, y_test_pred = round_prob(y_train_prob), round_prob(y_val_prob), round_prob(y_test_prob)
print("train accuracy:         ", accuracy_score(y_train_pred, y_train))
print("validation accuracy:    ", accuracy_score(y_val_pred, y_val))
print("validation precision:   ", np.round(precision_score(y_val, y_val_pred, average=None),4)) # true +ve / pred +ve
print("validation recall:      ", np.round(recall_score(y_val, y_val_pred, average=None),4))    # true +ve / all +ve
print("validation hamming loss:", hamming_loss(y_val_pred, y_val))

In [ ]:
import matplotlib.pyplot as plt
all_correct = (y_train_pred == y_train).all(axis=1)
counter = 0
fig, ax = plt.subplots(1,10, figsize=(12,8))
layer1, layer2 = model.layers[0:2]

for i, c in enumerate(all_correct):
    if not c:
        ax[counter].imshow(X_train[i,:,:,0])
        # ax2[counter].imshow(layer2(layer1(X_train[i:i+1]))[0,:,:,0])
        print("{}: ".format(i), y_train[i], "\n   ", y_train_pred[i],"\n   ", np.round(y_train_prob[i],2))
        counter += 1
    if counter == 10:
        break

In [ ]:
import pandas as pd
pd.DataFrame(y_test_prob1).to_csv("./output1.csv", index=False, header=False)